In [1]:
# default_exp products

In [2]:
#hide
!pip install -q nbdev lambdasdk s3bz

In [3]:
#hide
import pickle
KEY = ''
PW = ''
keypath = '/Users/nic/.villa-master-tester'
if KEY and PW:
  with open (keypath, 'wb') as f:
    pickle.dump({
        'KEY': KEY,
        'PW': PW
    }, f)
with open(keypath, 'rb') as f:
  creden = pickle.load(f)
USER = creden['KEY']
PW = creden['PW']

# Products

> interact with product apis

In [4]:
#hide
from nbdev.showdoc import *
import logging

In [5]:
#export
from botocore.config import Config
from s3bz.s3bz import S3, Requests
from lambdasdk.lambdasdk import Lambda
from villaProductSdk.schema import Event, Response
from typing import Optional
import ujson as json
import bz2,  boto3, base64, logging

In [6]:
#hide
# logging.basicConfig(level=logging.INFO)

In [7]:
#export
class FunctionNames:
  '''determine function and resources name based on branchName'''
  def __init__(self, branchName:str = 'dev-manual'):
    self.branchName = branchName
  dumpToS3 = lambda self: f'product-dump-s3-{self.branchName}'
  updateProduct = lambda self: f'product-update-{self.branchName}'
  updateS3 = lambda self: f'product-update-s3-{self.branchName}'
  singleQuery = lambda self: f'product-get-{self.branchName}'
  allQuery = lambda self: f'product-get-all-{self.branchName}'
  inputBucket = lambda self: f'input-product-bucket-{self.branchName}'
  inventoryBucket = lambda self: f'product-bucket-{self.branchName}'
  

In [8]:
#export
class ProductSdk:
  '''
    the main class for interacting with product endpoint
    user/pw are optional
  '''
  def __init__(self, 
               branch:str = 'dev-manual', 
               user:Optional[str] = None, 
               pw:Optional[str] = None,
               region:str = 'ap-southeast-1'):
    self.branchName = branch
    self.functionNames = FunctionNames(branchName = branch)
    self.lambdaClient = Lambda(user =user, pw=pw, region = region)
    self.user = user; self.pw = pw; self.region = region
    
  @staticmethod
  def returnLambdaResponse(lambdaResponse:dict):
    try:
      return Response.fromDict(lambdaResponse).body
    except:
      logging.exception(f'error parsing body, perhaps there is no body in response\
      response is {lambdaResponse}')
      logging.error(lambdaResponse)
  @staticmethod
  def printFirst(inputDict:dict):
    return next(iter(inputDict.items()))
    
  def updateWithS3(self, data, 
                   inputKeyName = 'input-data-name', 
                   invocationType = 'RequestResponse',
                   user= None, pw= None):
    # put users if not specified
    user = user or self.user; pw = pw or self.pw
    
    # extract function name and inputbucket name
    inputBucketName = self.functionNames.inputBucket()
    functionName = self.functionNames.updateS3()
    logging.info(f'bucket is {inputBucketName}')
    
    # save data to s3
    S3.save(key = inputKeyName, 
            objectToSave = data , 
            bucket = inputBucketName,
            user=user, pw=pw)
    logging.info(f'data is saved to s3, invoking ingestion function')
    
    # call lambda function
    inputValue = Event(body = json.dumps({ 'key': inputKeyName })).to_dict()
    logging.info(f'input to lambda is {inputValue}')
    lambdaResponse = self.lambdaClient.invoke(
      functionName= functionName ,
      input=inputValue, 
      invocationType= invocationType )
    logging.info(f'lambdaResponse is {lambdaResponse}')
    return self.returnLambdaResponse(lambdaResponse)

  def querySingleProduct(self, iprcode = '0171670', user=None, pw=None):
    '''query a single product'''
    #extract function name
    functionName = self.functionNames.singleQuery()
    query = {'iprcode': iprcode}
    inputValue = Event(body = json.dumps(query)).to_dict()
    logging.info(inputValue)
    lambdaResponse = self.lambdaClient.invoke(
        functionName = functionName , input = inputValue )
    return self.returnLambdaResponse(lambdaResponse)
  
  def allQuery(self):
    functionName = self.functionNames.allQuery()
    lambdaResponse = self.lambdaClient.invoke(
      functionName = functionName, input = {}
    )
    url = Response.fromDict(lambdaResponse).body['url']
    result = Requests.getContentFromUrl(url)
    return result
  def syncS3(self):
    '''force s3 to sync with the newly input data'''
    functionName = self.functionNames.dumpToS3()
    lambdaResponse = self.lambdaClient.invoke(
      functionName = functionName, input = {}
    )
    return self.returnLambdaResponse(lambdaResponse)


# Testing

In [9]:
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from random import randrange
from datetime import datetime
from pprint import pprint
import boto3


## generate dummy data for testing

In [10]:
#Dummy Data
sampleProducts = [{'cprcode': '0171670', 'iprcode': '0171670', 'oprcode': '0171670', 'ordertype': 'Y', 'pr_abb': 'JIRAPAT YOUG KALE 2', 'pr_active': 'Y', 'pr_cgcode': '05', 'pr_code': '0171670', 'pr_dpcode': '19', 'pr_engname': 'JIRAPAT YOUNG KALE 200 G.', 'pr_ggcode': '057', 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM845     ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0235141', 'iprcode': '0235141', 'oprcode': '0235141', 'ordertype': 'Y', 'pr_abb': 'EEBOO-PZCT3-PUZZLE', 'pr_active': 'Y', 'pr_cgcode': '08', 'pr_code': '0235141', 'pr_dpcode': '19', 'pr_engname': 'EEBOO,ANIMAL COUNTING PUZZLE_3ED,PZCT3', 'pr_ggcode': '113', 'pr_market': 'eeboo,PUZZLE-PZCT3', 'pr_name': 'EEBOO-PZCT3-ตัวต่อนับเลข ANIMAL COUNTING_3ED', 'pr_puqty': '1', 'pr_sa_method': '1', 'pr_sucode1': 'CM1979    ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0217153', 'iprcode': '0217153', 'oprcode': '0217153', 'ordertype': 'Y', 'pr_abb': 'COCOA LOCO MILK CHOC', 'pr_active': 'Y', 'pr_cgcode': '98', 'pr_code': '0217153', 'pr_dpcode': '28', 'pr_engname': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.', 'pr_ggcode': '003', 'pr_market': 'COCOA LOCO MILK CHOCOLATE OWL', 'pr_name': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.', 'pr_puqty': '24', 'pr_sa_method': '1', 'pr_sucode1': 'F1222     ', 'pr_suref3': 'S', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0182223', 'iprcode': '0182223', 'oprcode': '0182223', 'ordertype': 'Y', 'pr_abb': 'CIRIO PIZZASSIMO 400', 'pr_active': 'Y', 'pr_cgcode': '06', 'pr_code': '0182223', 'pr_dpcode': '06', 'pr_engname': 'CIRIO PIZZASSIMO 400G.', 'pr_ggcode': '004', 'pr_market': 'CIRIO ซอสทำพิซซ่า 400 G.', 'pr_name': 'CIRIO ซอสทำพิซซ่า 400 G.', 'pr_puqty': '12', 'pr_sa_method': '1', 'pr_sucode1': '2589      ', 'pr_suref3': 'C', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}, {'cprcode': '0124461', 'iprcode': '0124461', 'oprcode': '0124461', 'ordertype': 'Y', 'pr_abb': 'NEW CHOICE LYCHEE', 'pr_active': 'Y', 'pr_cgcode': '02', 'pr_code': '0124461', 'pr_dpcode': '02', 'pr_engname': 'NEW CHOICE LYCHEE', 'pr_ggcode': '003', 'pr_market': 'NEW CHOICE กลิ่นลิ้นจี่', 'pr_name': 'NEW CHOICE กลิ่นลิ้นจี่', 'pr_puqty': '12', 'pr_sa_method': '1', 'pr_sucode1': '695       ', 'pr_suref3': 'A', 'prtype': 'I', 'psqty': '1', 'pstype': '1'}]


## Create main class object

In [11]:
sdk = ProductSdk(branch = 'dev-manual', user=USER, pw=PW)

## Upload s3 data

In [12]:
#hide

In [13]:
%%time
result = sdk.updateWithS3(
    sampleProducts,
    invocationType = 'RequestResponse'
  )
result

ERROR:root:error parsing body, perhaps there is no body in response      response is {'errorMessage': "invalid literal for int() with base 10: ''", 'errorType': 'ValueError', 'stackTrace': ['  File "/var/task/villaProductDatabase/database.py", line 144, in lambdaUpdateS3\n    updateResult = ProductDatabase.updateS3Input(\n', '  File "/var/task/villaProductDatabase/update.py", line 81, in updateS3Input\n    updateResult = cls.valueUpdate(s3Result)\n', '  File "/var/task/villaProductDatabase/update.py", line 62, in valueUpdate\n    itemsUpdated[\'timetaken\'] = (datetime.now()- t0).total_seconds()*1000\n', '  File "/opt/python/pynamodb/models.py", line 113, in __exit__\n    return self.commit()\n', '  File "/opt/python/pynamodb/models.py", line 131, in commit\n    data = self.model._get_connection().batch_write_item(\n', '  File "/opt/python/pynamodb/connection/table.py", line 173, in batch_write_item\n    return self.connection.batch_write_item(\n', '  File "/opt/python/pynamodb/connect

CPU times: user 74.9 ms, sys: 10.1 ms, total: 85 ms
Wall time: 1.02 s


## Query Single Product

In [14]:
%%time
result = sdk.querySingleProduct('0171670')
result

CPU times: user 3.45 ms, sys: 1.14 ms, total: 4.58 ms
Wall time: 88 ms


{'cprcode': '0171670',
 'iprcode': '0171670',
 'oprcode': '0171670',
 'ordertype': 'Y',
 'pr_abb': 'JIRAPAT YOUNG KALE 2',
 'pr_active': 'Y',
 'pr_cgcode': '05',
 'pr_code': '0171670',
 'pr_dpcode': '19',
 'pr_engname': 'JIRAPAT YOUNG KALE 200 G.',
 'pr_ggcode': '057',
 'pr_market': 'JIRAPAT ยอดคะน้า 200 G.',
 'pr_name': 'JIRAPAT ยอดคะน้า 200 G.',
 'pr_puqty': '1',
 'pr_sa_method': '1',
 'pr_sucode1': 'CM845',
 'pr_suref3': 'A',
 'prtype': 'I',
 'psqty': '1',
 'pstype': '1'}

## All Query

In [15]:
result = sdk.allQuery()
sdk.printFirst(result)

('0217153',
 {'0217153': {'cprcode': '0217153',
   'iprcode': '0217153',
   'oprcode': '0217153',
   'ordertype': 'Y',
   'pr_abb': 'COCOA LOCO MILK CHOC',
   'pr_active': 'Y',
   'pr_cgcode': '98',
   'pr_code': '0217153',
   'pr_dpcode': '28',
   'pr_engname': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.',
   'pr_ggcode': '003',
   'pr_market': 'COCOA LOCO MILK CHOCOLATE OWL',
   'pr_name': 'COCOA LOCO MILK CHOCOLATE OWL LOLLY 26G.',
   'pr_puqty': '24',
   'pr_sa_method': '1',
   'pr_sucode1': 'F1222',
   'pr_suref3': 'S',
   'prtype': 'I',
   'psqty': '1',
   'pstype': '1'}})

## Trigger s3 sync

In [16]:
%%time
response = sdk.syncS3()
response

CPU times: user 2.68 ms, sys: 942 µs, total: 3.62 ms
Wall time: 394 ms


{'result': 'saved 5 products'}

In [17]:
#hide
from nbdev.export import *
notebook2script()

Converted Schema.ipynb.
Converted index.ipynb.
Converted product-sdk.ipynb.
